In [1]:
import json, re, unicodedata
import pandas as pd
from statistics import median
from pathlib import Path
from collections import Counter

cur_dir = Path(__name__).parent.resolve()

annotated_data = pd.read_csv(cur_dir.parent / "gemma_annotation.csv", dtype=str)

# ------------------ helper functions ------------------
def _nfkc(s: str) -> str:
    return unicodedata.normalize("NFKC", s)

def num(x):
    """Parse numbers robustly: NFKC, comma decimals, ASCII/Unicode fractions, '1½' & '1 ½', tolerate U+2044."""
    if x is None or (isinstance(x, float) and pd.isna(x)): return None
    s = _nfkc(str(x)).strip().lower()
    s = s.replace(",", ".").replace("⁄", "/")
    s = re.sub(r"(\d)([½¼¾⅓⅔⅛⅜⅝⅞])", r"\1 \2", s)    # "1½" -> "1 ½"
    s = re.split(r"\s+[a-zäöüß]+", s, maxsplit=1)[0]  # keep leading numeric chunk
    UF = {"½":0.5,"¼":0.25,"¾":0.75,"⅓":1/3,"⅔":2/3,"⅛":0.125,"⅜":0.375,"⅝":0.625,"⅞":0.875}
    total, seen = 0.0, False
    for tok in s.split():
        if tok in UF:
            total += UF[tok]; seen = True; continue
        if re.fullmatch(r"\d+/\d+", tok):
            n,d = tok.split("/")
            try: total += float(n)/float(d); seen = True; continue
            except: break
        try:
            total += float(tok); seen = True; continue
        except:
            break
    return total if seen else None

def clean_unit(u):
    if u is None or (isinstance(u, float) and pd.isna(u)): return ""
    return _nfkc(str(u)).lower().strip().replace(".","").replace("⁄","/")

def parse_ann(raw):
    if isinstance(raw, dict): return raw
    if not isinstance(raw, str): return None
    try:
        return json.loads(raw)
    except Exception:
        try:
            return json.loads(raw.replace('""','"'))
        except Exception:
            return None

def pick_amount(d):
    for k in ("gewicht","volumen","menge","anzahl"):
        v = num(d.get(k))
        if v is not None: return k, v
    return None, None

# ------------------ canonical targets ------------------
CANON_G, CANON_L, CANON_P = "g", "liter", "stück"

# Base map (extended later with tokens actually present)
BASE = {
    "g": (1.0, CANON_G), "kg": (1000.0, CANON_G), "mg": (0.001, CANON_G),
    "l": (1.0, CANON_L), "ml": (0.001, CANON_L), "cl": (0.01, CANON_L),
    "stk": (1.0, CANON_P), "stück": (1.0, CANON_P), "stueck": (1.0, CANON_P), "piece": (1.0, CANON_P)
}

# ------------------ mine unit-like tokens after numbers from `amount` ------------------
VULGAR = "½¼¾⅓⅔⅛⅜⅝⅞"
NUM = rf"(?:\d+[.,]?\d*|\d+/\d+|[{VULGAR}])"
RANGE_SEP = r"(?:\s*(?:-|–|—|to|bis)\s*)"
NUMRANGE = rf"{NUM}(?:{RANGE_SEP}{NUM})?"
WORD = r"[a-zA-ZäöüÄÖÜß]+\.?"
SYMBOL = r"[%%°]+"
pat_space = re.compile(rf"{NUMRANGE}\s*({WORD}|{SYMBOL})", re.IGNORECASE)
pat_glued = re.compile(rf"({NUM})([a-zA-ZäöüÄÖÜß]+\.?)", re.IGNORECASE)

def token_after_number(text: str):
    if not isinstance(text, str) or not text.strip(): return None
    t = _nfkc(text).strip().replace(",", ".").replace("⁄", "/")
    t = re.sub(rf"(\d)([{VULGAR}])", r"\1 \2", t)
    m = pat_space.search(t)
    if m: return m.group(1).lower().rstrip(".")
    m2 = pat_glued.search(t)  # search catches "ca. 200g", "≈200g"
    if m2: return m2.group(2).lower().rstrip(".")
    return None

present_tokens_amount = set()
token_counter = Counter()
for raw in annotated_data.get("amount", pd.Series([], dtype=str)).astype(str):
    tok = token_after_number(raw)
    if tok:
        present_tokens_amount.add(tok)
        token_counter[tok] += 1

# Also collect units that appear inside annotations
present_units_ann = set()
for raw in annotated_data.get("ingr_annotation", pd.Series([], dtype=str)).astype(str):
    ann = parse_ann(raw) if isinstance(raw, str) else None
    if ann:
        present_units_ann.add(clean_unit(ann.get("einheit")))

present_tokens_all = present_tokens_amount | present_units_ann

# ------------------ alias tables (used ONLY if token is present) ------------------
MASS_ALIAS = {
    "gram":"g","grams":"g","gramm":"g","gramme":"g","grammes":"g","gr":"g","g.":"g"
}
VOLUME_ALIAS = {
    "liter":"l","litre":"l","liters":"l","litres":"l","ltr":"l","lt":"l","l.":"l",
    "milliliter":"ml","millilitre":"ml","㎖":"ml","mℓ":"ml","mililiter":"ml"
}
PIECE_ALIAS = {
    "st":"stk","st.":"stk","pcs":"stk","pc":"stk","pieces":"stk","stuck":"stk",
    # additions seen
    "stck":"stk","stücke":"stk"
}

# Ambiguous *volume* units from our dataset → got it from tokens_present.csv
AMB_ALIASES = {
    "el": {"el","esslöffel","essloeffel"},
    "tl": {"tl","teelöffel","teeloeffel"},
    "tasse": {"tasse","tassen"},
    "becher": {"becher"},
    "glas": {"glas","gläser","glaeser"},
    "prise": {"prise"},
    "msp": {"msp","messerspitze"},
    "schuss": {"schuss"},
    "spritzer": {"spritzer", "spr"},
    "tropfen": {"tropfen"},
    "bestecklöffel": {"bestecklöffel"},
}
DEFAULT_ML = {
    "el": 15, "tl": 5, "tasse": 250, "becher": 250, "glas": 200, "prise": 0.5, "msp": 0.5,
    "schuss": 20,          # ~10–20 ml; set 20 ml
    "spritzer": 2,         # small splash
    "tropfen": 0.05,       # per drop
    "bestecklöffel": 10,   # between TL (5) and EL (15)
}

# Packaging/count (non-liquid ambiguous) → pieces (only if present)
PIECEY_ALIASES = {
    "päckchen": {"päckchen","packchen","päck.","päck","packung","pkg"},
    "dose": {"dose","dosen"},
    "flasche": {"flasche","flaschen"},
    "bund": {"bund","bunde"},
    "kopf": {"kopf","köpfe","koepfe"},
    "scheibe": {"scheibe","scheiben"},
    "zehe": {"zehe","zehen"},
    "stange": {"stange","stangen"},
    "beutel": {"beutel"},
    "tüte": {"tüte","tuete"},
    # NEW from your tokens
    "blatt": {"blatt","blätter"},
    "bündel": {"bündel","bd"},
    "frucht": {"frucht"},
    "hand": {"hand"},
    "stiel": {"stiel"},
    "würfel": {"würfel"},
    "zweig": {"zweig","zweige"},
    "knolle": {"knolle"},
    "pack": {"pack"},
    "schüssel": {"schüssel"},
}

def extend_base_from_present(base: dict, tokens: set):
    for t, tgt in MASS_ALIAS.items():
        if t in tokens: base[t] = base.get(tgt, (1.0, CANON_G))
    for t, tgt in VOLUME_ALIAS.items():
        if t in tokens: base[t] = base.get(tgt, (1.0, CANON_L) if tgt=="l" else (0.001, CANON_L))
    for t, tgt in PIECE_ALIAS.items():
        if t in tokens: base[t] = base.get(tgt, (1.0, CANON_P))
    # packaging/count → pieces (only if present)
    for canon, aliases in PIECEY_ALIASES.items():
        if any(a in tokens for a in aliases) or canon in tokens:
            for a in aliases | {canon}:
                if a in tokens: base[a] = (1.0, CANON_P)

extend_base_from_present(BASE, present_tokens_all)

# ambiguous volume units we will use (present and defaults)
def canon_amb(u_raw: str) -> str:
    u = clean_unit(u_raw)
    for c, aliases in AMB_ALIASES.items():
        if u == c or u in aliases: return c
    return u

amb_from_amount = {c for c, aliases in AMB_ALIASES.items()
                   if any(tok == c or tok in aliases for tok in present_tokens_amount)}
amb_from_ann = {canon_amb(u) for u in present_units_ann}
AMB_USED = sorted((amb_from_amount | amb_from_ann) & set(DEFAULT_ML.keys()))

# summary (defaults only)
def count_in_amount_for(canon: str) -> int:
    aliases = AMB_ALIASES.get(canon, {canon})
    return sum(token_counter.get(a, 0) for a in (aliases | {canon}))

def count_in_ann_for(canon: str) -> int:
    cnt = 0
    for raw in annotated_data.get("ingr_annotation", pd.Series([], dtype=str)).astype(str):
        ann = parse_ann(raw)
        if not ann: continue
        if canon_amb(ann.get("einheit")) == canon:
            cnt += 1
    return cnt

summary_rows, ML_PER = [], {}
for u in AMB_USED:
    chosen = DEFAULT_ML[u]
    ML_PER[u] = chosen
    summary_rows.append({
        "unit": u,
        "evidence_n": 0,
        "chosen_ml_per_unit": chosen,
        "source": "default_fallback",
        "count_in_amount": count_in_amount_for(u),
        "count_in_annotation": count_in_ann_for(u)
    })

df_summary = pd.DataFrame(summary_rows).sort_values("unit")
df_summary.to_csv("ambiguous_summary.csv", index=False, encoding="utf-8")

# Also dump the plain token counts we saw in `amount`
pd.DataFrame(sorted(token_counter.items(), key=lambda x: (-x[1], x[0])),
             columns=["token","count"]).to_csv("tokens_present.csv", index=False, encoding="utf-8")

# And a simple presence report for ambiguous units (even if not in AMB_USED)
presence_rows = []
for u, aliases in AMB_ALIASES.items():
    presence_rows.append({
        "unit": u,
        "present_in_amount": int(any(a in present_tokens_amount for a in (aliases|{u}))),
        "present_in_annotation": int(any(canon_amb(x)==u for x in present_units_ann)),
        "default_ml": DEFAULT_ML[u]
    })
pd.DataFrame(presence_rows).sort_values("unit").to_csv("ambiguous_presence.csv", index=False, encoding="utf-8")

# ------------------ build unit map (defaults for AMB_USED) ------------------
unit_map = dict(BASE)
for k, ml in ML_PER.items():  
    unit_map[k] = (ml/1000.0, CANON_L)

# ------------------ normalize rows ------------------
vals, units = [], []
for _, row in annotated_data.iterrows():
    try:
        ann = parse_ann(row.get("ingr_annotation"))
        if not ann: raise ValueError("no_json")
        key, amt = pick_amount(ann)
        if amt is None:
            amt = num(row.get("amount"))
            if amt is None: amt = 1.0
        u = canon_amb(clean_unit(ann.get("einheit")))
        if u in unit_map:
            f, target = unit_map[u]
            val, unit = float(amt)*f, target
        else:
            if key in ("menge","anzahl"):   val, unit = float(amt), CANON_P
            elif key == "gewicht":          val, unit = float(amt), CANON_G
            elif key == "volumen":          val, unit = float(amt), CANON_L
            else:                           val, unit = float(amt), CANON_P
    except Exception:
        v = num(row.get("amount"))
        val, unit = (v if v is not None else 1.0), CANON_P
    vals.append(val); units.append(unit)

annotated_data["norm_value"] = vals
annotated_data["norm_unit"]  = units

# ------------------ duplicates diagnostics ------------------
dup_subset = [c for c in ["ingredient","amount","ingr_annotation"] if c in annotated_data.columns]
if dup_subset:
    dup_rows = annotated_data[annotated_data.duplicated(subset=dup_subset, keep=False)].copy()
    dup_rows = dup_rows.sort_values(dup_subset)
    dup_rows.to_csv("duplicates.csv", index=False, encoding="utf-8")

    dup_groups = (annotated_data
                  .groupby(dup_subset, dropna=False)
                  .size().reset_index(name="count")
                  .query("count > 1")
                  .sort_values("count", ascending=False))
    dup_groups.to_csv("duplicates_groups.csv", index=False, encoding="utf-8")

# ------------------ ingredient-based profiling (soft duplicates) ------------------
def get_ing_key(row):
    # Prefer annotated ingredient name if present; else fall back to raw text
    ann_amt = parse_ann(row.get("amount_annotation"))
    name = (ann_amt or {}).get("zutat")
    if not name:
        ann_ing = parse_ann(row.get("ingr_annotation"))
        name = (ann_ing or {}).get("zutat") or row.get("ingredient", "")
    return _nfkc(str(name)).lower().strip()

def get_ann_key(row):
    ann = parse_ann(row.get("ingr_annotation"))
    if not ann: return None
    for k in ("gewicht","volumen","menge","anzahl"):
        if ann.get(k) is not None:
            return k
    return None

def get_ann_unit(row):
    ann = parse_ann(row.get("ingr_annotation"))
    return clean_unit(ann.get("einheit")) if ann else ""

df = annotated_data.copy()
df["ing_key"]   = df.apply(get_ing_key, axis=1)
df["ann_key"]   = df.apply(get_ann_key, axis=1)      # which JSON field carried the amount
df["ann_unit"]  = df.apply(get_ann_unit, axis=1)     # raw unit from JSON (cleaned)

def counter_to_sorted_dict(c: Counter):
    return dict(sorted(c.items(), key=lambda kv: (-kv[1], kv[0])))

profiles = []
for ing, grp in df.groupby("ing_key", dropna=False):
    profiles.append({
        "ingredient": ing,
        "n_rows": len(grp),
        "n_amount_unique": int(grp["amount"].nunique() if "amount" in grp else 0),
        "ann_key_counts": counter_to_sorted_dict(Counter(grp["ann_key"].fillna("None"))),
        "ann_unit_counts": counter_to_sorted_dict(Counter([u for u in grp["ann_unit"] if u])),
        "norm_unit_counts": counter_to_sorted_dict(Counter(grp["norm_unit"])),
    })

profiles_df = pd.DataFrame(profiles).sort_values(["ingredient","n_rows"], ascending=[True, False])
profiles_df.to_csv("ingredient_profiles.csv", index=False, encoding="utf-8")

conflicts = []
for ing, grp in df.groupby("ing_key", dropna=False):
    nu = Counter(grp["norm_unit"])
    if len(nu) > 1:
        total = sum(nu.values())
        top_unit, top_cnt = max(nu.items(), key=lambda kv: kv[1])
        conflicts.append({
            "ingredient": ing,
            "norm_unit_counts": counter_to_sorted_dict(nu),
            "majority_unit": top_unit,
            "majority_share": round(top_cnt / total, 3),
            "n_rows": total
        })
conflicts_df = pd.DataFrame(conflicts).sort_values(["majority_share","n_rows"], ascending=[True, False])
conflicts_df.to_csv("ingredient_conflicts.csv", index=False, encoding="utf-8")

consensus = []
for ing, grp in df.groupby("ing_key", dropna=False):
    nu = Counter(grp["norm_unit"])
    total = sum(nu.values())
    top_unit, top_cnt = max(nu.items(), key=lambda kv: kv[1])
    consensus.append({
        "ingredient": ing,
        "suggested_unit": top_unit,
        "support": top_cnt,
        "support_share": round(top_cnt / total, 3),
        "n_rows": total
    })
consensus_df = pd.DataFrame(consensus).sort_values(["support_share","n_rows"], ascending=[False, False])
consensus_df.to_csv("unit_consensus.csv", index=False, encoding="utf-8")

#  preview & save
print("Present tokens from amount:", sorted(present_tokens_amount))
print("Present units from annotation:", sorted(present_units_ann))
print(f"BASE extended with present aliases. Keys: {len(BASE)}")
print(f"Ambiguous VOLUME units considered (present ∩ defaults): {AMB_USED}")
print("Wrote: ambiguous_summary.csv, ambiguous_presence.csv, tokens_present.csv")
if dup_subset:
    print("Wrote: duplicates.csv, duplicates_groups.csv")
print("Wrote: ingredient_profiles.csv, ingredient_conflicts.csv, unit_consensus.csv")

try:
    display(df_summary.head(12))
    display(annotated_data[["ingredient","amount","ingr_annotation","norm_value","norm_unit"]].head(60))
    display(consensus_df.head(20))
    display(conflicts_df.head(20))
except NameError:
    pass

NORMALIZED_CSV = cur_dir.parent / "gemma_annotation_normalized.csv"
annotated_data.to_csv(NORMALIZED_CSV, index=False, encoding="utf-8")


Present tokens from amount: ['bd', 'becher', 'bestecklöffel', 'beutel', 'blatt', 'blätter', 'bund', 'bündel', 'cl', 'dose', 'dosen', 'el', 'esslöffel', 'frucht', 'g', 'glas', 'gr', 'gramm', 'hand', 'kg', 'kl', 'klecks', 'kleine', 'kleinere', 'knolle', 'kopf', 'l', 'lange', 'liter', 'mal', 'mg', 'mittelgross', 'mittelgrosse', 'ml', 'msp', 'pack', 'pck', 'prise', 'päckchen', 'scheibe', 'scheiben', 'schuss', 'schüssel', 'spr', 'spritzer', 'stange', 'stangen', 'stck', 'stiel', 'stk', 'stück', 'stücke', 'tasse', 'tassen', 'teelöffel', 'tl', 'tropfen', 'voll', 'würfel', 'zehe', 'zehen', 'zweig', 'zweige']
Present units from annotation: ['bd', 'becher', 'bestecklöffel', 'beutel', 'blatt', 'bund', 'bündel', 'cl', 'dose', 'dosen', 'el', 'esslöffel', 'frucht', 'g', 'glas', 'gr', 'gramm', 'hand', 'hand voll', 'kg', 'klecks', 'knolle', 'kopf', 'l', 'limette', 'liter', 'mal', 'mg', 'ml', 'msp', 'pack', 'pck', 'prise', 'päckchen', 'scheibe', 'scheiben', 'schuss', 'schüssel', 'spritzer', 'stange', 's

,unit,evidence_n,chosen_ml_per_unit,source,count_in_amount,count_in_annotation
0,becher,0,250.00,default_fallback,7,7
1,bestecklöffel,0,10.00,default_fallback,2,2
2,el,0,15.00,default_fallback,135,135
3,glas,0,200.00,default_fallback,1,1
4,msp,0,0.50,default_fallback,2,2
5,prise,0,0.50,default_fallback,37,39
6,schuss,0,20.00,default_fallback,5,5
7,spritzer,0,2.00,default_fallback,3,2
8,tasse,0,250.00,default_fallback,6,7
9,tl,0,5.00,default_fallback,73,73


,ingredient,amount,ingr_annotation,norm_value,norm_unit
0,zucchini,1,"{""anzahl"": 1, ""einheit"": ""St\u00fcck""}",1.0000,stück
1,paprika rot,2,"{""anzahl"": 2, ""einheit"": ""St\u00fcck""}",2.0000,stück
2,feta,200 g,"{""gewicht"": 200, ""einheit"": ""g""}",200.0000,g
3,schinken,250 g,"{""gewicht"": 250, ""einheit"": ""g""}",250.0000,g
4,zwiebel frisch,1,"{""anzahl"": 1, ""einheit"": ""St\u00fcck""}",1.0000,stück
5,passierte tomaten,300 g,"{""gewicht"": 300, ""einheit"": ""g""}",300.0000,g
6,hähnchenbrust frisch,500 g,"{""gewicht"": 500, ""einheit"": ""g""}",500.0000,g
7,zwiebel frisch,120 g,"{""gewicht"": 120, ""einheit"": ""g""}",120.0000,g
8,austernpilze,300 g,"{""gewicht"": 300, ""einheit"": ""g""}",300.0000,g
9,schmand,1 becher,"{""volumen"": 1, ""einheit"": ""becher""}",0.2500,liter


,ingredient,suggested_unit,support,support_share,n_rows
283,wasser,liter,28,1.0,28
63,essig,liter,16,1.0,16
295,zitrone,stück,9,1.0,9
296,zitronensaft,liter,9,1.0,9
29,butterschmalz,liter,8,1.0,8
158,milch,liter,8,1.0,8
240,senf,liter,8,1.0,8
291,zehe,stück,8,1.0,8
124,knoblauchzehe,stück,7,1.0,7
266,teelöffel,liter,7,1.0,7


,ingredient,norm_unit_counts,majority_unit,majority_share,n_rows
24,käse,"{'stück': 3, 'g': 2, 'liter': 2}",stück,0.429,7
35,petersilie,"{'liter': 3, 'stück': 3}",stück,0.500,6
41,rucola,"{'g': 3, 'stück': 3}",g,0.500,6
1,basilikum,"{'liter': 2, 'stück': 2}",stück,0.500,4
5,camembert,"{'g': 2, 'stück': 2}",stück,0.500,4
51,speisestärke,"{'g': 2, 'liter': 2}",liter,0.500,4
54,staudensellerie,"{'g': 2, 'stück': 2}",g,0.500,4
2,blätterteig,"{'g': 1, 'stück': 1}",g,0.500,2
3,bohne,"{'g': 1, 'liter': 1}",g,0.500,2
7,chicorée,"{'g': 1, 'stück': 1}",g,0.500,2
